In [2]:
from pyquil import Program, get_qc
import qulacs as q
from qulacs.gate import *
from qulacs import PauliOperator
from qulacs.quantum_operator import create_quantum_operator_from_openfermion_file
from qulacs.quantum_operator import create_quantum_operator_from_openfermion_text
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.transforms import get_sparse_operator
from qulacs.observable import create_split_observable, create_observable_from_openfermion_text
from qulacs import Observable, QuantumStateGpu, QuantumCircuit
import pyquil.api as api
import numpy as np
import math

The process of creating a hamiltonian in qulacs is much less straightforward than with pyquil. Qulacs uses openfermion text to create the hamiltonian, which is why this cell block is much longer. It can also be read in from a text file.

In [3]:
# hopping term: a_p^dagger a_q
U1 = 1
U2 = 0
e1 = 2
e2 = 0
n_qubits = 4
depth = 4
hamt_text = """
(-0.5 + 0j) [X0 X1] + 
(-0.5 + 0j) [X2 X3] + 
(-0.5 + 0j) [Y0 Y1] + 
(-0.5 + 0j) [Y2 Y3]
"""

hamt = create_quantum_operator_from_openfermion_text(hamt_text)

# interaction term: a_p^dagger a_q^dagger a_q a_p
hamu_text = """
(0.25 + 0j) [I3] - 
(0.25 + 0j) [Z0] - 
(0.25 + 0j) [Z2] + 
(0.25 + 0j) [Z0 Z2] + 
(0.0 + 0j) [I0] - 
(0.0 + 0j) [Z1] - 
(0.0 + 0j) [Z3] +
(0.0 + 0j) [Z1 Z3]
"""
hamU = create_quantum_operator_from_openfermion_text(hamu_text)

# number term: a_p^dagger a_p
hame_text = """
(2.0 + 0j) [I] - 
(1.0 + 0j) [Z0] - 
(1.0 + 0j) [Z2] - 
(0.0 + 0j) [I] -
(0.0 + 0j) [Z1] -
(0.0 + 0j) [Z3]
"""
hame = create_quantum_operator_from_openfermion_text(hame_text)

hamiltonian_text = """
(-0.5 + 0j) [X0 X1] + 
(-0.5 + 0j) [X2 X3] + 
(-0.5 + 0j) [Y0 Y1] + 
(-0.5 + 0j) [Y2 Y3]
(0.25 + 0j) [I3] - 
(0.25 + 0j) [Z0] - 
(0.25 + 0j) [Z2] + 
(0.25 + 0j) [Z0 Z2] + 
(0.0 + 0j) [I0] - 
(0.0 + 0j) [Z1] - 
(0.0 + 0j) [Z3] +
(0.0 + 0j) [Z1 Z3]
(2.0 + 0j) [I] - 
(1.0 + 0j) [Z0] - 
(1.0 + 0j) [Z2] - 
(0.0 + 0j) [I] -
(0.0 + 0j) [Z1] -
(0.0 + 0j) [Z3]
"""
hamiltonian = create_observable_from_openfermion_text(hamiltonian_text)
print(hamiltonian)

In [ ]:
"""
from grove.pyvqe.vqe import VQE
from scipy.optimize import minimize
import numpy as np

vqe_inst = VQE(minimizer=minimize, minimizer_kwargs={'method': 'nelder-mead'})
"""

In [ ]:
"""from qucochemistry.utils import pyquilpauli_to_qubitop
from openfermion.transforms import get_sparse_operator
h = get_sparse_operator(pyquilpauli_to_qubitop(hamiltonian))
np.savetxt('h.txt', np.real(h.todense()), '%-2d', ', ')
if h.shape[0] > 1024:
    [w, _] = sp.sparse.linalg.eigsh(h, k=1)
else:
    [w, _] = np.linalg.eigh(h.todense())
w
"""

The ansatz program is mostly the same as with pyquil when creating the circuit. 

In [12]:
 def var_ansatz(params):
    S = 2
    p = Program()
    
    # put initial state in the sector
    
    # Product state -- 1/sqrt(2) (|1001> + |0110>)
    c = QuantumCircuit(4)
    c.add_H_gate(0)
    c.add_CNOT_gate(0, 1)
    c.add_CNOT_gate(1, 2)
    c.add_X_gate(1)
    c.add_CNOT_gate(2, 3)
    c.add_X_gate(2)
    # |1001>
    #p.inst(X(0))
    #p.inst(X(3))
    
    # HF state -- 1/2 (|0101> + |0110> + |1001> + |1010>)

    for i in range(S):
        # building U_u
        c.add_RZ_gate(0, U1*params[3*i]/4)
        c.add_RZ_gate(1, U2*params[3*i]/4)
        c.add_RZ_gate(2, U1*params[3*i]/4)
        c.add_RZ_gate(3, U2*params[3*i]/4)
        c.add_CNOT_gate(0, 2)
        c.add_RZ_gate(2, -U1*params[3*i]/4)
        c.add_CNOT_gate(0, 2)
        c.add_CNOT_gate(1, 3)
        c.add_RZ_gate(3, -U2*params[3*i]/4)
        c.add_CNOT_gate(1, 3)
        
        #building U_\epsilon
        c.add_RZ_gate(1, e2*params[3*i+2])
        c.add_RZ_gate(3, e2*params[3*i+2])
        c.add_RZ_gate(0, e1*params[3*i+2])
        c.add_RZ_gate(2, e1*params[3*i+2])
        
        # building U_t
        c.add_H_gate(0)
        c.add_H_gate(1)
        c.add_H_gate(2)
        c.add_H_gate(3)
        c.add_CNOT_gate(0, 1)
        c.add_CNOT_gate(2, 3)
        c.add_RZ_gate(1, params[3*i+1])
        c.add_RZ_gate(3, params[3*i+1])
        c.add_CNOT_gate(0, 1)
        c.add_CNOT_gate(2, 3)
        c.add_H_gate(0)
        c.add_H_gate(1)
        c.add_H_gate(2)
        c.add_H_gate(3)
        c.add_RX_gate(0, -np.pi/2)
        c.add_RX_gate(1, -np.pi/2)
        c.add_RX_gate(2, -np.pi/2)
        c.add_RX_gate(3, -np.pi/2)
        c.add_CNOT_gate(0, 1)
        c.add_CNOT_gate(2, 3)
        c.add_RZ_gate(1, params[3*i+1])
        c.add_RZ_gate(3, params[3*i+1])
        c.add_CNOT_gate(0, 1)
        c.add_CNOT_gate(2, 3)
        c.add_RX_gate(0, -np.pi/2)
        c.add_RX_gate(1, -np.pi/2)
        c.add_RX_gate(2, -np.pi/2)
        c.add_RX_gate(3, -np.pi/2)
        
        # building U_u
        c.add_RZ_gate(0, U1*params[3*i]/4)
        c.add_RZ_gate(1, U1*params[3*i]/4)
        c.add_RZ_gate(2, U1*params[3*i]/4)
        c.add_RZ_gate(3, U1*params[3*i]/4)
        c.add_CNOT_gate(0, 2)
        c.add_RZ_gate(2, -U1*params[3*i]/4)
        c.add_CNOT_gate(0, 2)
        c.add_CNOT_gate(1, 3)
        c.add_RZ_gate(3, -U2*params[3*i]/4)
        c.add_CNOT_gate(1, 3)
        
    return c

Here I define the VQE with qulacs version. It is a lot different than with pyquil/grove, and I am not sure how to specify the optimization method yet to nelder-mead, which may be part of why the results are different.

In [16]:
def cost(x):
    state = QuantumStateGpu(n_qubits)
    circuit = var_ansatz(x)
    circuit.update_quantum_state(state)
    return hamiltonian.get_expectation_value(state)

This code is mostly the same as the other H2VQE code, with the adjustments for qulacs' VQE.

In [14]:
from scipy.optimize import minimize
import random

def expectation(x):
    """
    Expectation value for parameters, x
    """
    return cost(x)

def scipy_minimize(x):
    """
    Minimizes expectation value using parameters found in greedy search as starting point
    Powell's conjugate direction method
    """
    return minimize(expectation, x, method='powell')

def greedy_noisy_search(initial_state):
    """
    Slightly perturb the values of the points, accepting whenever this results in a lower energy
    Total of 150 evaulations of the energy
    Change step size after 30 evaluations based on number of acceptances in previous trial group
    """
    params = initial_state
    min_energy = cost(initial_state)

    def random_point(dim, step):
        """
        Generates a random point on the perimeter of a circle with radius, step
        """
        coords = [random.gauss(0, 1) for i in range(dim)]
        norm = math.sqrt(sum([i**2 for i in coords]))
        coords = [(i / norm) * step for i in coords]
        return coords

    step = 0.1
    acceptances = 0

    # Five groups of 30 trials
    for i in range(5):
        acceptances = 0
        for j in range(30):
            
            # Slightly perturb the values of the points
            coords = random_point(len(initial_state), step)
            temp_params = [sum(x) for x in zip(params, coords)]
            
            # Calculate expectation value with new parameters
            temp_energy = cost(temp_params)
            print(temp_params)
            
            # Greedily accept parameters that result in lower energy
            if (temp_energy < min_energy):
                min_energy = temp_energy
                params = temp_params
                acceptances += 1
        # Update step size
        step *= (acceptances / 15)

    return {'x': params, 'energy': min_energy, 'step': step}

def global_variational(S):
    """
    Complete a round of greedy and Powell's optimization for six randomly chosen points
    Further optimizes the best point
    """    
    points = [np.random.rand(S*3)*0.2 for i in range(6)]
    greedy_results = [greedy_noisy_search(x) for x in points]
    print("Done with greedy search")
    results = [scipy_minimize(i['x']) for i in greedy_results]
    print("Done with Powell's")
    res = min(results, key=lambda x:x.fun)
    print(res)
    # For the chosen point, we continue optimizing until we cannot find improvement
    res_copy = res
    energy = res_copy.fun
    while (True):
        result = greedy_noisy_search(res_copy.x)
        temp_res = scipy_minimize(result['x'])
        temp_energy = temp_res.fun
        print(temp_energy)

        # Optimizing will usually find an energy ~1e-6 lower
        # Stop it eventually
        tolerance = 0.00001
        if (abs(temp_energy - energy) > tolerance):
            res_copy = temp_res
            energy = temp_energy
        else:
            break
    return res_copy

In [15]:
# make sure to change this S to match the one in the ansatz
S = 2
res = global_variational(S)
print(res)

[0.1724535050779595, 0.1076516740479512, 0.08438578081058998, 0.14403738081800327, 0.1246124481673154, -0.02259124515095337]
[0.1106247497014955, 0.025325455730916522, 0.09465041207040403, 0.14298020740924827, 0.06936767374898423, 0.044046586314869476]
[0.214467033444397, 0.05289689549846001, 0.15242245853409933, 0.16037319811130285, 0.17544863035698366, -0.004197777202422419]
[0.19170289838862212, 0.06771741315331359, 0.015607863594969629, 0.14693143081635332, 0.12156466183490534, 0.021613468208259063]
[0.26447171298073174, 0.04125685248382994, 0.14400487779246202, 0.22310606806667782, 0.1261257797088539, 0.0636297943728624]
[0.10950590694427415, 0.0797632100738967, 0.11229683260686027, 0.21406083651813093, 0.12880243295792299, -0.004147448783552443]
[0.2122516448047705, -0.004862195825359954, 0.1137896319193916, 0.14736223662494163, 0.13067927935153295, -0.0247721212860071]
[0.141978779991854, -0.006756770973402708, 0.08455681456520789, 0.18122661065295304, 0.07057179440910588, -0.01

[0.09095600623159454, 0.0732918889825967, 0.07090198692783038, 0.15458910111270066, 0.18857673059867863, 0.07356583486838533]
[0.10413599007609076, 0.07058456279347039, 0.10256647660011613, 0.033554637654007194, 0.060038338921225315, 0.07094918338547657]
[0.08992079316278047, 0.06266250262891591, -0.019192590796158406, 0.02884352958042355, 0.12475950798570892, 0.10533827444427843]
[0.08871887288011224, 0.03648847479680339, 0.07521747703060758, 0.14500706386441856, 0.15389939129674676, 0.11523472772271498]
[0.0866568764679798, 0.12953303744796962, 0.06473734433134834, 0.09476555757415121, 0.11808363712493078, 0.13582351546586152]
[0.11584398857118551, 0.12134764250775583, 0.10135602863977027, 0.034827342667511335, 0.0837350878901892, 0.06536905721674639]
[0.12041622385361082, 0.1293684959817895, 0.1031391614106078, 0.09542347633709294, 0.15080598556344743, 0.10169435828299922]
[0.1154374234495678, 0.10028336783665313, 0.04204238269787306, 0.07734495033949461, 0.11266995269794317, 0.1498

[0.09505038867646697, 0.0719595824768144, 0.04673516193018539, 0.08555137468660666, 0.1225278943176845, 0.057171655500536225]
[0.09505038867646697, 0.0719595824768144, 0.04673516193018539, 0.08555137468660666, 0.1225278943176845, 0.057171655500536225]
[0.09505038867646697, 0.0719595824768144, 0.04673516193018539, 0.08555137468660666, 0.1225278943176845, 0.057171655500536225]
[0.09505038867646697, 0.0719595824768144, 0.04673516193018539, 0.08555137468660666, 0.1225278943176845, 0.057171655500536225]
[0.09505038867646697, 0.0719595824768144, 0.04673516193018539, 0.08555137468660666, 0.1225278943176845, 0.057171655500536225]
[0.09505038867646697, 0.0719595824768144, 0.04673516193018539, 0.08555137468660666, 0.1225278943176845, 0.057171655500536225]
[0.09505038867646697, 0.0719595824768144, 0.04673516193018539, 0.08555137468660666, 0.1225278943176845, 0.057171655500536225]
[0.09505038867646697, 0.0719595824768144, 0.04673516193018539, 0.08555137468660666, 0.1225278943176845, 0.057171655500

[0.1645553578685888, 0.16862984736263167, 0.19165526626056437, 0.10901971684896208, 0.041600319220443605, 0.07658655726876591]
[0.1645553578685888, 0.16862984736263167, 0.19165526626056437, 0.10901971684896208, 0.041600319220443605, 0.07658655726876591]
[0.1645553578685888, 0.16862984736263167, 0.19165526626056437, 0.10901971684896208, 0.041600319220443605, 0.07658655726876591]
[0.1645553578685888, 0.16862984736263167, 0.19165526626056437, 0.10901971684896208, 0.041600319220443605, 0.07658655726876591]
[0.1645553578685888, 0.16862984736263167, 0.19165526626056437, 0.10901971684896208, 0.041600319220443605, 0.07658655726876591]
[0.1645553578685888, 0.16862984736263167, 0.19165526626056437, 0.10901971684896208, 0.041600319220443605, 0.07658655726876591]
[0.1645553578685888, 0.16862984736263167, 0.19165526626056437, 0.10901971684896208, 0.041600319220443605, 0.07658655726876591]
[0.1645553578685888, 0.16862984736263167, 0.19165526626056437, 0.10901971684896208, 0.041600319220443605, 0.076

[0.09757503425924603, 0.03436566657516662, 0.048329277633587436, 0.068302853459955, 0.0756573692899672, 0.19029390689921033]
[0.09757503425924603, 0.03436566657516662, 0.048329277633587436, 0.068302853459955, 0.0756573692899672, 0.19029390689921033]
[0.09757503425924603, 0.03436566657516662, 0.048329277633587436, 0.068302853459955, 0.0756573692899672, 0.19029390689921033]
[0.09757503425924603, 0.03436566657516662, 0.048329277633587436, 0.068302853459955, 0.0756573692899672, 0.19029390689921033]
[0.09757503425924603, 0.03436566657516662, 0.048329277633587436, 0.068302853459955, 0.0756573692899672, 0.19029390689921033]
[0.09757503425924603, 0.03436566657516662, 0.048329277633587436, 0.068302853459955, 0.0756573692899672, 0.19029390689921033]
[0.09757503425924603, 0.03436566657516662, 0.048329277633587436, 0.068302853459955, 0.0756573692899672, 0.19029390689921033]
[0.09757503425924603, 0.03436566657516662, 0.048329277633587436, 0.068302853459955, 0.0756573692899672, 0.19029390689921033]


[0.09115051401738371, 0.0037337664524257352, 0.11677331439416004, 0.12156168124321341, 0.027400359331144042, 0.04224346676716315]
[0.09115051401738371, 0.0037337664524257352, 0.11677331439416004, 0.12156168124321341, 0.027400359331144042, 0.04224346676716315]
[0.09115051401738371, 0.0037337664524257352, 0.11677331439416004, 0.12156168124321341, 0.027400359331144042, 0.04224346676716315]
[0.09115051401738371, 0.0037337664524257352, 0.11677331439416004, 0.12156168124321341, 0.027400359331144042, 0.04224346676716315]
[0.09115051401738371, 0.0037337664524257352, 0.11677331439416004, 0.12156168124321341, 0.027400359331144042, 0.04224346676716315]
[0.09115051401738371, 0.0037337664524257352, 0.11677331439416004, 0.12156168124321341, 0.027400359331144042, 0.04224346676716315]
[0.09115051401738371, 0.0037337664524257352, 0.11677331439416004, 0.12156168124321341, 0.027400359331144042, 0.04224346676716315]
[0.09115051401738371, 0.0037337664524257352, 0.11677331439416004, 0.12156168124321341, 0.0

Done with Powell's
   direc: array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])
     fun: array(0.)
 message: 'Optimization terminated successfully.'
    nfev: 73
     nit: 1
  status: 0
 success: True
       x: array([2.77356305, 2.63704784, 2.69077045, 2.77381065, 2.70083556,
       2.62992563])
[2.7557232512349565, 2.6870490768532727, 2.6752947950528667, 2.744577893581515, 2.6753582834157252, 2.5561796003971198]
[2.778418664444266, 2.593907760876908, 2.657031816887882, 2.8318334225928172, 2.755611255897317, 2.6546246718064266]
[2.791093848307193, 2.6745199667465362, 2.7359890757555014, 2.8488260510661774, 2.721223394130322, 2.6440964822946653]
[2.7435068819689667, 2.587074011887594, 2.6766174089002748, 2.7188354995426636, 2.6505414582599607, 2.659031807880422]
[2.756429166849887, 2.608459455983225, 2.7139114811121283, 2.709193780615092, 

[2.7735630541722203, 2.637047843015599, 2.690770446358428, 2.773810653364573, 2.7008355621147775, 2.6299256259720307]
[2.7735630541722203, 2.637047843015599, 2.690770446358428, 2.773810653364573, 2.7008355621147775, 2.6299256259720307]
[2.7735630541722203, 2.637047843015599, 2.690770446358428, 2.773810653364573, 2.7008355621147775, 2.6299256259720307]
[2.7735630541722203, 2.637047843015599, 2.690770446358428, 2.773810653364573, 2.7008355621147775, 2.6299256259720307]
[2.7735630541722203, 2.637047843015599, 2.690770446358428, 2.773810653364573, 2.7008355621147775, 2.6299256259720307]
[2.7735630541722203, 2.637047843015599, 2.690770446358428, 2.773810653364573, 2.7008355621147775, 2.6299256259720307]
[2.7735630541722203, 2.637047843015599, 2.690770446358428, 2.773810653364573, 2.7008355621147775, 2.6299256259720307]
[2.7735630541722203, 2.637047843015599, 2.690770446358428, 2.773810653364573, 2.7008355621147775, 2.6299256259720307]
[2.7735630541722203, 2.637047843015599, 2.69077044635842

The output here is the part that I am having trouble understanding. I am not sure what the nfev and nit variables represent. I believe the x array are the energy values, which are different than the pyquil output. Assuming all is running correctly, the execution of the global_variational function is much quicker when running on the GPU, which is a good sign. However, the output appears to be the ground state, so I think that it is not working.

In [19]:
print(res)

   direc: array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])
     fun: array(0.)
 message: 'Optimization terminated successfully.'
    nfev: 73
     nit: 1
  status: 0
 success: True
       x: array([2.77356305, 2.63704784, 2.69077045, 2.77381065, 2.70083556,
       2.62992563])


I have not gotten to redefining the rest of the code yet, as I want to make sure everything up to this point is running correctly.

In [18]:
def op(s1, s2):
    # returns a PauliSum representing a_s1^daggar a_s2
    def sigma_plus(s):
        # returns sigma^+ operator on side s
        # sigma^+ = 1/2(X + iY)
        return PauliTerm('X', s, 0.5) + PauliTerm('Y', s, complex(0.0+0.5j))
    def sigma_minus(s):
        # returns sigma^- operator on side s
        # sigma^- = 1/2(X - iY)
        return PauliTerm('X', s, 0.5) - PauliTerm('Y', s, complex(0.0+0.5j))
    
    # for all sites in between site one (s1) and site two (s2), multiply by sigma z
    z = sI()
    for i in range(s1+1, s2):
        z *= PauliTerm('Z', i)
        
    return sigma_minus(s1) * z * sigma_plus(s2)